In [1]:
using OrderConditions

It holds
$$\left(\frac{d}{dt}\right)^k\mathrm{e}^{tB(t)} = \mathcal{C}_k(t)\mathrm{e}^{tB(t)},$$
where $C_k$ satisfies the recursion
$$\mathcal{C}_0(t)=\mathrm{Id},\quad\mathcal{C}_1(t) = C(t),\qquad \mathcal{C}_k(t)=\mathcal{C}_{k-1}(t)\cdot C(t)+\frac{d}{dt}\mathcal{C}_{k-1}(t)$$
with
$$C(t) = B(t)+\sum_{k=0}^{\infty}\frac{t^{k+1}}{(k+1)!}\mathrm{ad}_{B(t)}(B'(t)).$$

The following function implements this recursion:

In [2]:
gen_exp_tBt_derivatives_coeffs(4)

5-element Array{Dict{Array{Int64,1},Int64},1}:
 Dict(Int64[]=>1)                                                                     
 Dict([0]=>1)                                                                         
 Dict([1]=>1,[0,0]=>1)                                                                
 Dict([0,1]=>1,[2]=>1,[1,0]=>2,[0,0,0]=>1)                                            
 Dict([3]=>1,[2,0]=>3,[0,0,1]=>1,[1,1]=>3,[0,1,0]=>2,[1,0,0]=>3,[0,0,0,0]=>1,[0,2]=>1)

Here, the keys Int64[], [0], [1], [0,0], [0,1], ... correspond respectively to  $\mathrm{Id}, C(t), C'(t), C(t)^2, C(t)\cdot C'(t),\dots$,
and the values are the corresponding coefficients,  
such that
(arguments of $\mathcal{C}_k(t), C(t), C'(t), \dots$ ommited)
$$\mathcal{C}_0 = \mathrm{Id},$$
$$\mathcal{C}_1 = C,$$
$$\mathcal{C}_2 = C^2+ C',$$
$$\mathcal{C}_3 = C^3 +2C'C+ CC'+C'',$$
$$\mathcal{C}_4 = C^4 +3C'C^2+C^2C'+2CC'C+3C''C+CC''+C'''.$$

It holds $C(0)=B(0)$, and for the derivatives of $C(t)$ evaluated at $t=0$ it holds
$$C^{(n)}(0) = B^{(n)}(0) +\sum_{k=0}^{n-1}{n \choose k+1}\left[\left(\frac{d}{dt}\right)^{n-k-1}\mathrm{ad}_{B(t)}^k(B'(t))\right]_{t=0},\quad n=1,2,\dots.$$
The following function computes these derivatives:

In [3]:
gen_C_derivatives_at_0(4)

5-element Array{Dict{Array{Int64,1},Int64},1}:
 Dict([0]=>1)                                                     
 Dict([1]=>2)                                                     
 Dict([0,1]=>1,[2]=>3)                                            
 Dict([3]=>4,[0,0,1]=>1,[0,2]=>3)                                 
 Dict([1,0,1]=>4,[0,3]=>6,[4]=>5,[1,2]=>6,[0,0,2]=>4,[0,0,0,1]=>1)

Here, the keys [0], [1], [0,1], [0,0,1], [0,2]... correspond respectively to iterated commutators 
$B(0), B'(0), [B(0),B'(0)], [B(0),[B(0),B'(0)]], [B(0),B''(0)],\dots$, and the values are the corresponding coefficients, such that (argument 0 ommited):
$$C=B,$$
$$C'=2B',$$
$$C''=3B''+[B,B'],$$
$$C'''=4B'''+3[B,B'']+[B,[B,B']],$$
$$C^{(4)}=5B^{(4)}+6[B',B'']+6[B,B''']+4[B',[B,B']]+4[B,[B,B'']]+[B,[B,[B,B']]].$$

In [4]:
using Giac

In [5]:
J, K, N = 2, 2, 4

(2,2,4)

Substitute 
$$B^{(n)}=b_nA^{(n)},\quad b_n=\sum_{l}a_lc_l^n$$
in the above equations for $C, C',C''',...$: 

In [6]:
b = giac[giac(string("b",k)) for k=0:N] 

5-element Array{Giac.giac,1}:
 b0
 b1
 b2
 b3
 b4

In [7]:
C = gen_C_derivatives_at_0_in_terms_of_A(N, b)
C = [Dict{Array{Int64,1},Giac.giac}([key=>factor(val) for (key,val) in c]) for c in C]

5-element Array{Dict{Array{Int64,1},Giac.giac},1}:
 Dict{Array{Int64,1},Giac.giac}([0]=>b0)                                                                                         
 Dict{Array{Int64,1},Giac.giac}([1]=>2*b1)                                                                                       
 Dict{Array{Int64,1},Giac.giac}([0,1]=>b1*b0,[2]=>3*b2)                                                                          
 Dict{Array{Int64,1},Giac.giac}([3]=>4*b3,[0,0,1]=>b0^2*b1,[0,2]=>3*b2*b0)                                                       
 Dict{Array{Int64,1},Giac.giac}([1,0,1]=>4*b1^2*b0,[0,3]=>6*b3*b0,[4]=>5*b4,[1,2]=>6*b2*b1,[0,0,2]=>4*b0^2*b2,[0,0,0,1]=>b0^3*b1)

Now the keys [0], [1], [0,1], [0,0,1], [0,2]... correspond respectively to iterated commutators 
$A(0), A'(0), [A(0),A'(0)], [A(0),[A(0),A'(0)]], [A(0),A''(0)],\dots$, and the values are the corresponding coefficients (which are Giac expressions), such that (argument 0 ommited):
$$C_j = b_0A,$$
$$C_j' = 2b_1 A',$$
$$C_j'' = 3b_2 A''+b_0b_1[A,A'],$$
$$C_j''' = 4b_3A''' +3b_0b_2[A,A'']+b_0^2b_1[A,[A,A']],$$
$$C^{(4)}=5b_4A^{(4)}+6b_1b_2[A',A'']+6b_0b_3[A,A''']+4b_1^2b_0[A',[A,A']]+4b_0^2b_2[A,[A,A'']]+b_0^3b_1[A,[A,[A,A']]].$$

Here we substitute $b_n=\sum_{l}a_lc_l^n$:

In [8]:
a = [giac[giac(string("a",j,k)) for k=1:K] for j=1:J];
c = giac[giac(string("c",k)) for k=1:K]; 

In [9]:
bb = giac[sum([a[1][j]*c[j]^n for j=1:J]) for n=0:N]

5-element Array{Giac.giac,1}:
 a11+a12          
 a11*c1+a12*c2    
 a11*c1^2+a12*c2^2
 a11*c1^3+a12*c2^3
 a11*c1^4+a12*c2^4

In [10]:
[Dict{Array{Int64,1},Giac.giac}([key=>subst(val,b,bb) for (key,val) in c]) for c in C]

5-element Array{Dict{Array{Int64,1},Giac.giac},1}:
 Dict{Array{Int64,1},Giac.giac}([0]=>a11+a12)                                                                                                                                                                                                                           
 Dict{Array{Int64,1},Giac.giac}([1]=>2*(a11*c1+a12*c2))                                                                                                                                                                                                                 
 Dict{Array{Int64,1},Giac.giac}([0,1]=>(a11*c1+a12*c2)*(a11+a12),[2]=>3*(a11*c1^2+a12*c2^2))                                                                                                                                                                            
 Dict{Array{Int64,1},Giac.giac}([3]=>4*(a11*c1^3+a12*c2^3),[0,0,1]=>(a11+a12)^2*(a11*c1+a12*c2),[0,2]=>3*(a11*c1^2+a12*c2^2)*(a11+a12))                   

Again, the keys [0], [1], [0,1], [0,0,1], [0,2]... correspond respectively to iterated commutators 
$A(0), A'(0), [A(0),A'(0)], [A(0),[A(0),A'(0)]], [A(0),A''(0)],\dots$, and the values are the corresponding coefficients (which are Giac expressions), such that (argument 0 ommited):
$$C_j = (a_{j1}+a_{j2})A,$$
$$C_j' = 2(a_{j1}c_1+a_{j2}c_2)A',$$
$$C_j'' = 3(a_{j1}c_1^2+a_{j2}c_2^2)A''+(a_{j1}+a_{j2})(a_{j1}c_1+a_{j2}c_2)[A,A']$$

Now we can compute
$$\left[\left(\frac{d}{dt}\right)^k\mathrm{e}^{tB(t)}\right]_{t=0} = \big[\mathcal{C}_{k}(t)\mathrm{e}^{tB(t)}\big]_{t=0}=\mathcal{C}_{k}(0),\quad k=1,2,\dots$$
with the $\mathcal{C}_k$ form above. For $k=0$ it holds $\left[\mathrm{e}^{tB(t)}\right]_{t=0}=\mathrm{Id}$ of course.

In [11]:
C = gen_exp_tBt_derivatives_at_0_in_terms_of_A(N, b)
C = [Dict{Array{Array{Int64,1},1},Giac.giac}([key=>factor(val) for (key,val) in c]) for c in C]

5-element Array{Dict{Array{Array{Int64,1},1},Giac.giac},1}:
 Dict{Array{Array{Int64,1},1},Giac.giac}([Int64[]]=>1)                                                                                                                                                                                                                                           
 Dict{Array{Array{Int64,1},1},Giac.giac}([[0]]=>b0)                                                                                                                                                                                                                                              
 Dict{Array{Array{Int64,1},1},Giac.giac}([[1]]=>2*b1,[[0],[0]]=>b0^2)                                                                                                                                                                                                                            
 Dict{Array{Array{Int64,1},1},Giac.giac}([[0,1]]=>b1*b0,[[2]]=>3*b2,[[

The output is to be interpreted as follows (argument 0 ommited):
$$\mathcal{C}_{0} = \mathrm{Id},$$
$$\mathcal{C}_{1} = b_0A,$$
$$\mathcal{C}_{2} = 2b_1A'+ b_0^2A^2,$$
$$\mathcal{C}_{3} = 3b_2A''+b_0b_1[A,A']+4b_0b_1A'A+2b_0b_1AA'+b_0^3A^3$$
$$\mathcal{C}_{4} = 4b_3A'''+3b_0b_2AA''+9b_0b_2A''A+3b_0b_2[A,A'']+4b_0^2b_1AA'A+6b_0^2b_1A'A^2+2b_0^2b_1A^2A'+12b_1^2(A')^2
+b_0^2b_1A[A,A']+3b_0^2b_1[A,A']A+b_0^2b_1[A,[A,A']]+b_0^4A^4.$$
Here it seems reasonable to expand all commutators, because the expanded terms are present in the expressions anyway.

In [12]:
C = [expand_commutators(c) for c in C]
C = [Dict{Array{Array{Int64,1},1},Giac.giac}([key=>factor(val) for (key,val) in c]) for c in C]

5-element Array{Dict{Array{Array{Int64,1},1},Giac.giac},1}:
 Dict{Array{Array{Int64,1},1},Giac.giac}([Int64[]]=>1)                                                                                                                                                           
 Dict{Array{Array{Int64,1},1},Giac.giac}([[0]]=>b0)                                                                                                                                                              
 Dict{Array{Array{Int64,1},1},Giac.giac}([[1]]=>2*b1,[[0],[0]]=>b0^2)                                                                                                                                            
 Dict{Array{Array{Int64,1},1},Giac.giac}([[1],[0]]=>3*b1*b0,[[2]]=>3*b2,[[0],[0],[0]]=>b0^3,[[0],[1]]=>3*b0*b1)                                                                                                  
 Dict{Array{Array{Int64,1},1},Giac.giac}([[0],[0],[0],[0]]=>b0^4,[[1],[0],[0]]=>4*b0^2*b1,[[0],[0],[

The output is to be interpreted as follows (argument 0 ommited):
$$\mathcal{C}_{0} = \mathrm{Id},$$
$$\mathcal{C}_{1} = b_0A,$$
$$\mathcal{C}_{2} = 2b_1A'+ b_0^2A^2,$$
$$\mathcal{C}_{3} = 3b_2A''+3b_0b_1A'A+3b_0b_1AA'+b_0^3A^3,$$
$$\mathcal{C}_{4} = 4b_3A'''+6b_0b_2A''A+6b_0b_2AA''+12b_1^2(A')^2+4b_0^2b_1A'A^2+4b_0^2b_1AA'A+4b_0^2b_1A^2A'+b_0^4A^4$$

Numerical method:
$$S(t) = \prod_{j=J,\dots,1}\mathrm{e}^{tB_j(t)}.$$
Defect:
$$D(t) = \frac{d}{dt}S(t)-A(t)S(t).$$
Derivatives of the defect are calculated using the Leibniz formula for higher derivatives:
$$D^{(q)}(t) = \sum_{\mathbf{k}=(k_1,\dots,k_J)\atop |\mathbf{k}|=q+1}{q+1 \choose \mathbf{k}}\prod_{j=J,\dots,1}\left(\frac{d}{dt}\right)^{k_j}\mathrm{e}^{tB_j(t)}-
\sum_{\mathbf{k}=(k_1,\dots,k_{J+1})\atop |\mathbf{k}|=q}{q \choose \mathbf{k}}A^{(k_{J+1})}(t)\prod_{j=J,\dots,1}\left(\frac{d}{dt}\right)^{k_j}\mathrm{e}^{tB_j(t)}.
$$

Substituting $\left(\frac{d}{dt}\right)^k\mathrm{e}^{tB_j(t)} = \mathcal{C}_{j,k}(t)\mathrm{e}^{tB_j(t)}$ followed
by an evaluation at $t=0$ yields
$$D^{(q)}(0) = \sum_{\mathbf{k}=(k_1,\dots,k_J)\atop |\mathbf{k}|=q+1}{q+1 \choose \mathbf{k}}\prod_{j=J,\dots,1}\mathcal{C}_{j,k_j}(0)-\sum_{\mathbf{k}=(k_1,\dots,k_{J+1})\atop |\mathbf{k}|=q}{q \choose \mathbf{k}}A^{(k_{J+1})}(0)\prod_{j=J,\dots,1}\mathcal{C}_{j,k_j}(0).$$